# Building an Agent with Custom Tools using Google ADK and Flotorch

This notebook demonstrates how to create a Google ADK agent and equip it with a set of custom tools. The agent will be powered by the Flotorch LLM and will be able to perform specific tasks like calculations, time-telling, and text analysis by using locally defined Python functions.

### Prerequesit
Configure model, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Define multiple Python functions to act as custom tools.
- Wrap these functions using Google ADK's `FunctionTool`.
- Configure an `LlmAgent` to use the Flotorch LLM and provide it with the custom tools.
- Use the `Runner` to process user queries and observe the agent's tool-using capabilities.

## 1. Environment Setup and Imports

The following cells handle the complete setup. This includes installing the `flotorch[adk]` package, defining API credentials, and importing the necessary components from Google ADK and the Flotorch library.

In [ ]:
# install flotorch adk package
%pip install flotorch[adk]

In [ ]:
FLOTORCH_API_KEY = "flotorch api key"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "flotorch model"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

In [ ]:
from datetime import datetime

# Import Flotorch and Google ADK components
from flotorch.adk.llm import FlotorchADKLLM
from google.adk.sessions import InMemorySessionService
from google.adk.agents import LlmAgent
from google.adk.tools import FunctionTool
from google.adk import Runner
from google.genai import types

print("Imported Required Libraries Successfully")

## 2. Defining Custom Tools

To extend the agent's capabilities, we define several Python functions. Each function is then wrapped using Google ADK's `FunctionTool` class. This makes the functions available to the agent, which can intelligently decide when to call them based on the user's query. The function's docstring is crucial as it provides the description the agent uses to understand the tool's purpose.

In [ ]:
def calculator(expression: str) -> str:
    """
    Calculate basic math expressions safely.
    
    Args:
        expression (str): Mathematical expression to evaluate
        
    Returns:
        str: Calculation result or error message
    """
    try:
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic math operations allowed"
        
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"


def get_current_time() -> str:
    """
    Get the current date and time.
    
    Returns:
        str: Formatted current date and time
    """
    now = datetime.now()
    return f"Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"


def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""

custom_tools = [
        FunctionTool(func=calculator),
        FunctionTool(func=get_current_time),
        FunctionTool(func=analyze_text)
    ]

print("Custom tools defined successfully.")

## 3. Configuring the LLM and Agent

With the tools defined, we now configure the agent. This involves two steps:
1. **LLM Configuration**: We instantiate `FlotorchADKLLM` to act as the agent's "brain".
2. **Agent Configuration**: We create an `LlmAgent`, providing it with a descriptive name, instructions, the LLM instance, and the list of `custom_tools` we just created.

In [ ]:
model =  FlotorchADKLLM(
        model_id= FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url= FLOTORCH_BASE_URL,
    )
    
print(f"Flotorch LLM model configured: {model.model}")

In [ ]:
agent = LlmAgent(
        name="Flotorch_Tool_Assistant",
        description="A helpful Flotorch AI assistant with calculator, time, and text analysis tools",
        instruction="""You are Flotorch AI assistant with special tools. You can:
        - Perform mathematical calculations using the calculator tool
        - Get the current date and time using the time tool
        - Analyze text and provide statistics using the text analyzer tool
        
        Use these tools when appropriate to provide accurate and helpful responses.""",
        model=model,
        tools=custom_tools,
    )


print(f"Agent created successfully: {agent.name}")
print(f"Available tools: {len(custom_tools)} custom tools")

## 4. Running the Agent

Finally, we set up the `Runner`, which orchestrates the entire process. The `Runner` takes the agent and a session service (`InMemorySessionService` for this example) to manage the conversation. The following cells demonstrate how to create a session and interact with the agent, which will now use its tools to answer relevant queries.

In [ ]:
# Initialize Session Service
session_service = InMemorySessionService()

# Initialize runner
runner =  Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("Runner configured successfully.")

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that request."

print("Message processing function defined.")

In [ ]:
async def chat_with_agent(query,session_id):
    """
    Chat function that takes a user query and returns the agent's response.
    
    Args:
        query (str): User's input message
        session_id : User's session ID
    Returns:
        str: Agent's response
    """
    # Get response from agent
    response = run_single_turn(query, session_id, USER_ID, runner)
    return response

print("Chat function ready. Use chat_with_agent('your query',session.id) to interact with the agent.")

In [ ]:
# Example: Calculate a mathematical expression
session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
response = await chat_with_agent("What's 15 * 7?",session.id)
print(f"Query: What's 15 * 7?")
print(f"Response: {response}")

In [ ]:
# Example: Get current time
response = await chat_with_agent("What time is it?",session.id)
print(f"Query: What time is it?")
print(f"Response: {response}")

In [ ]:
# Example: Analyze text
response = await chat_with_agent("Analyze this text: Flotorch makes AI development easy!",session.id)
print(f"Query: Analyze this text: Flotorch makes AI development easy!")
print(f"Response: {response}")

## Summary

This notebook demonstrated the complete process of building a tool-augmented agent using Google ADK and Flotorch. We successfully extended a basic LLM agent with custom Python functions, enabling it to perform specific, real-world tasks.

### Key Achievements:

- **Tool Creation**: We defined three distinct Python functions for calculation, time-telling, and text analysis.
- **Tool Integration**: These functions were seamlessly integrated into the `LlmAgent` using Google ADK's `FunctionTool` wrapper.
- **Intelligent Execution**: The agent demonstrated its ability to understand user intent and autonomously select the correct tool to fulfill the request.
- **Flexible Framework**: This example highlights the flexibility of the ADK framework for creating powerful, specialized AI assistants.